In [183]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

## Chargement des données en Data Frame

In [184]:
train_home = pd.read_csv('Datas/X_Train_Data/train_home_team_statistics_df.csv')
train_away = pd.read_csv('Datas/X_Train_Data/train_away_team_statistics_df.csv')
y_train = pd.read_csv('Datas/Y_train_1rknArQ.csv')

#Test data
test_home = pd.read_csv('Datas/X_Test_Data/test_home_team_statistics_df.csv')
test_away = pd.read_csv('Datas/X_Test_Data/test_away_team_statistics_df.csv')
y_test = pd.read_csv('Datas/Y_test_random_sEE2QeA.csv')

# y_train.head()
test_away.head()
# train_away.head()






,ID,TEAM_SHOTS_TOTAL_season_sum,TEAM_SHOTS_INSIDEBOX_season_sum,TEAM_SHOTS_OFF_TARGET_season_sum,TEAM_SHOTS_ON_TARGET_season_sum,TEAM_SHOTS_OUTSIDEBOX_season_sum,TEAM_PASSES_season_sum,TEAM_SUCCESSFUL_PASSES_season_sum,TEAM_SAVES_season_sum,TEAM_CORNERS_season_sum,...,TEAM_YELLOWCARDS_5_last_match_std,TEAM_REDCARDS_5_last_match_std,TEAM_OFFSIDES_5_last_match_std,TEAM_ATTACKS_5_last_match_std,TEAM_PENALTIES_5_last_match_std,TEAM_SUBSTITUTIONS_5_last_match_std,TEAM_BALL_SAFE_5_last_match_std,TEAM_DANGEROUS_ATTACKS_5_last_match_std,TEAM_INJURIES_5_last_match_std,TEAM_GOALS_5_last_match_std
0,12303,6.0,6.0,5.0,7.0,7.0,4.0,3.0,6.0,3.0,...,6.0,0.0,1.0,4.0,0.0,8.0,7.0,2.0,7.0,0.0
1,12304,4.0,4.0,4.0,4.0,3.0,8.0,8.0,4.0,7.0,...,0.0,0.0,3.0,6.0,8.0,5.0,1.0,1.0,7.0,9.0
2,12305,1.0,1.0,2.0,2.0,2.0,6.0,5.0,4.0,2.0,...,10.0,0.0,0.0,2.0,0.0,4.0,2.0,3.0,NaN,9.0
3,12306,6.0,9.0,6.0,9.0,4.0,4.0,4.0,1.0,10.0,...,1.0,5.0,4.0,5.0,10.0,9.0,6.0,9.0,7.0,3.0
4,12307,5.0,5.0,2.0,4.0,4.0,1.0,2.0,3.0,8.0,...,3.0,0.0,6.0,5.0,0.0,4.0,3.0,10.0,3.0,1.0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV


# 1. Nettoyage des colonnes non numériques
train_home_clean = train_home.drop(columns=["LEAGUE", "TEAM_NAME"])
train_away_clean = train_away.drop(columns=["LEAGUE", "TEAM_NAME"])
# test_home_clean = test_home.drop(columns=["LEAGUE", "TEAM_NAME"])
# test_away_clean = test_away.drop(columns=["LEAGUE", "TEAM_NAME"])

# 2. Préfixe pour éviter conflits de noms

train_home_clean.columns = ["HOME_" + col for col in train_home_clean.columns]
train_away_clean.columns = ["AWAY_" + col for col in train_away_clean.columns]

test_home.columns = ["HOME_" + col for col in test_home.columns]
test_away.columns = ["AWAY_" + col for col in test_away.columns]

# Vérifie si les colonnes ont déjà été préfixées

# 3. Fusion des données
X_train = pd.concat([train_home_clean, train_away_clean], axis=1)
X_test = pd.concat([test_home, test_away], axis=1)

# X_train.head(10)
# X_test.head(10)




# 4. Remplissage des valeurs manquantes
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# 5. Création des labels
y_labels = y_train[["HOME_WINS", "DRAW", "AWAY_WINS"]].idxmax(axis=1).map({
    "AWAY_WINS": 0,
    "DRAW": 1,
    "HOME_WINS": 2
})

y_test_labels = y_test[["HOME_WINS", "DRAW", "AWAY_WINS"]].idxmax(axis=1).map({
    "AWAY_WINS": 0,
    "DRAW": 1,
    "HOME_WINS": 2
})

# 6. Entraînement du modèle
# from sklearn.ensemble import GradientBoostingClassifier
# model = GradientBoostingClassifier()

model = RandomForestClassifier(random_state=42, n_estimators=1000, max_depth=20, min_samples_split=2, criterion="gini" )
model.fit(X_train, y_labels)

# 7. Prédictions
y_pred = model.predict(X_test)




# 8. Formatage des prédictions en one-hot
pred_df = pd.DataFrame(0, index=range(len(y_pred)), columns=["HOME_WINS", "DRAW", "AWAY_WINS"])
for i, pred in enumerate(y_pred):
    if pred == 2:
        pred_df.loc[i, "HOME_WINS"] = 1
    elif pred == 1:
        pred_df.loc[i, "DRAW"] = 1
    else:
        pred_df.loc[i, "AWAY_WINS"] = 1

# 9. Ajout des IDs
pred_df["ID"] = y_test["ID"]
pred_df = pred_df[["ID", "HOME_WINS", "DRAW", "AWAY_WINS"]].sort_values("ID").reset_index(drop=True)

# 10. Export CSV
pred_df.to_csv("match_predictions.csv", index=False)
print("Fichier 'match_predictions.csv'' généré avec succès.")



Fichier 'match_predictions.csv'' généré avec succès.'
